## One time data transformation
In this notebook, we are going to transform the stations and weather data in such a way that they will be conformed to the redshift schema for their corresponding tables.

The preprocessed data will be saved back to S3 before getting loaded to Redshift.

In [1]:
import pyspark
import os

In [2]:
pyspark.__version__

'3.2.1'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
        .master('local[*]') \
        .appName('data-transformer') \
        .config("spark.hadoop.fs.s3a.access.key", os.environ.get('AWS_ACCESS_KEY'))\
        .config("spark.hadoop.fs.s3a.secret.key", os.environ.get('AWS_SECRET_ACCESS_KEY'))\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/05 23:56:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sc = spark.sparkContext
sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

'3.3.1'

###  1. Stations data

In [6]:
df_stations = spark.read.csv("s3a://hrc-de-data/raw/cycling-extras/stations.csv", inferSchema=True, header=True)

22/03/01 18:05:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
df_stations.take(2)

[Row(Station.Id=1, StationName='River Street, Clerkenwell', longitude=-0.109971, latitude=51.5292, Easting=531202.52, Northing=182832.02),
 Row(Station.Id=2, StationName='Phillimore Gardens, Kensington', longitude=-0.197574, latitude=51.4996, Easting=525207.07, Northing=179391.86)]

In [8]:
df_stations.printSchema()

root
 |-- Station.Id: integer (nullable = true)
 |-- StationName: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- Easting: double (nullable = true)
 |-- Northing: double (nullable = true)



In [11]:
from pyspark.sql import functions as F, types as T

In [10]:
# rename columns
stations= df_stations.withColumnRenamed('Station.Id', 'station_id') \
                        .withColumnRenamed('StationName', 'station_name') \
                        .withColumnRenamed('easting', 'easting') \
                        .withColumnRenamed('northing', 'northing') 

In [11]:
stations.show(5)

+----------+--------------------+----------+--------+----------+----------+
|station_id|        station_name| longitude|latitude|   easting|  northing|
+----------+--------------------+----------+--------+----------+----------+
|         1|River Street, Cle...| -0.109971| 51.5292| 531202.52| 182832.02|
|         2|Phillimore Garden...| -0.197574| 51.4996| 525207.07| 179391.86|
|         3|Christopher Stree...|-0.0846057| 51.5213| 532984.81| 182001.53|
|         4|St. Chad's Street...| -0.120974| 51.5301| 530436.76| 182911.99|
|         5|Sedding Street, S...| -0.156876| 51.4931|528051.649|178742.097|
+----------+--------------------+----------+--------+----------+----------+
only showing top 5 rows



In [12]:
# count missing values in each column
stations.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in stations.columns]
   ).show()

+----------+------------+---------+--------+-------+--------+
|station_id|station_name|longitude|latitude|easting|northing|
+----------+------------+---------+--------+-------+--------+
|         0|           0|        0|       0|      0|       0|
+----------+------------+---------+--------+-------+--------+



In [17]:
stations.write.parquet('s3a://hrc-de-data/processed/cycling-dimension/stations/', mode='overwrite')

### 2. Weather data

In [6]:
df_weather = spark.read.json("s3a://hrc-de-data/raw/cycling-extras/weather.json")

22/03/05 23:57:03 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
df_weather.take(2)

22/03/05 23:57:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(cloudcover=0.5, conditions='Rain', datetime='2021-01-01', datetimeEpoch=1609459200, description='Clear conditions throughout the day with late afternoon rain.', dew=0.8, feelslike=-0.2, feelslikemax=2.9, feelslikemin=-3.6, humidity=91.03, icon='rain', moonphase=0.53, precip=0.22, precipcover=4.17, precipprob=None, preciptype=None, pressure=1011.6, severerisk=None, snow=None, snowdepth=None, solarenergy=0.8, solarradiation=29.4, source='obs', stations=['03769099999', '03680099999', 'D5621', '03672099999', '03781099999', '03772099999', '03770099999'], sunrise='08:06:14', sunriseEpoch=1609488374, sunset='16:02:22', sunsetEpoch=1609516942, temp=2.1, tempmax=5.0, tempmin=-0.5, tzoffset=None, uvindex=0.0, visibility=2.6, winddir=304.0, windgust=None, windspeed=6.6),
 Row(cloudcover=0.5, conditions='Rain', datetime='2021-01-02', datetimeEpoch=1609545600, description='Clear conditions throughout the day with rain.', dew=1.0, feelslike=1.5, feelslikemax=3.1, feelslikemin=-1.5, humidity=82.

In [8]:
df_weather.printSchema()

root
 |-- cloudcover: double (nullable = true)
 |-- conditions: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- datetimeEpoch: long (nullable = true)
 |-- description: string (nullable = true)
 |-- dew: double (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- feelslikemax: double (nullable = true)
 |-- feelslikemin: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- icon: string (nullable = true)
 |-- moonphase: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- precipcover: double (nullable = true)
 |-- precipprob: double (nullable = true)
 |-- preciptype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pressure: double (nullable = true)
 |-- severerisk: double (nullable = true)
 |-- snow: double (nullable = true)
 |-- snowdepth: double (nullable = true)
 |-- solarenergy: double (nullable = true)
 |-- solarradiation: double (nullable = true)
 |-- source: string (nullable = true)
 |-- s

In [9]:
# drop some columns that we won't need
weather= df_weather.drop('cloudcover', 'conditions', 'datetimeEpoch', 'description', 'dew', 'icon', 
                            'precipcover', 'preciptype', 'source', 'stations', 'sunriseEpoch', 'sunsetEpoch')

In [12]:
# transform datetime
weather= weather.withColumnRenamed('datetime', 'weather_date') 
weather= weather.withColumn('weather_date', weather.weather_date.cast(T.DateType()))

In [13]:
weather.printSchema()
print(len(weather.columns), 'columns')

root
 |-- weather_date: date (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- feelslikemax: double (nullable = true)
 |-- feelslikemin: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- moonphase: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- precipprob: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- severerisk: double (nullable = true)
 |-- snow: double (nullable = true)
 |-- snowdepth: double (nullable = true)
 |-- solarenergy: double (nullable = true)
 |-- solarradiation: double (nullable = true)
 |-- sunrise: string (nullable = true)
 |-- sunset: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- tempmax: double (nullable = true)
 |-- tempmin: double (nullable = true)
 |-- tzoffset: double (nullable = true)
 |-- uvindex: double (nullable = true)
 |-- visibility: double (nullable = true)
 |-- winddir: double (nullable = true)
 |-- windgust: double (nullable = true)
 |-- windspeed: double (

In [18]:
weather.show(2)

+------------+---------+------------+------------+--------+---------+------+----------+--------+----------+----+---------+-----------+--------------+--------+--------+----+-------+-------+--------+-------+----------+-------+--------+---------+
|weather_date|feelslike|feelslikemax|feelslikemin|humidity|moonphase|precip|precipprob|pressure|severerisk|snow|snowdepth|solarenergy|solarradiation| sunrise|  sunset|temp|tempmax|tempmin|tzoffset|uvindex|visibility|winddir|windgust|windspeed|
+------------+---------+------------+------------+--------+---------+------+----------+--------+----------+----+---------+-----------+--------------+--------+--------+----+-------+-------+--------+-------+----------+-------+--------+---------+
|  2021-01-01|     -0.2|         2.9|        -3.6|   91.03|     0.53|  0.22|      null|  1011.6|      null|null|     null|        0.8|          29.4|08:06:14|16:02:22| 2.1|    5.0|   -0.5|    null|    0.0|       2.6|  304.0|    null|      6.6|
|  2021-01-02|      1.5|

In [27]:
# count missing values in windgust
missing_windgust= (
    weather.select(
        F.count(F.when(F.col('tzoffset').isNull() | F.isnan(F.col('tzoffset')), ''))
        .alias('missing_tzoffset'))
)
missing_windgust.show()

+----------------+
|missing_tzoffset|
+----------------+
|             179|
+----------------+



In [25]:
# count missing values in each column
cols= weather.columns
cols.remove('weather_date')
missing_values= weather.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c)).alias(c) for c in cols])

In [26]:
missing_values.show()

+---------+------------+------------+--------+---------+------+----------+--------+----------+----+---------+-----------+--------------+-------+------+----+-------+-------+--------+-------+----------+-------+--------+---------+
|feelslike|feelslikemax|feelslikemin|humidity|moonphase|precip|precipprob|pressure|severerisk|snow|snowdepth|solarenergy|solarradiation|sunrise|sunset|temp|tempmax|tempmin|tzoffset|uvindex|visibility|winddir|windgust|windspeed|
+---------+------------+------------+--------+---------+------+----------+--------+----------+----+---------+-----------+--------------+-------+------+----+-------+-------+--------+-------+----------+-------+--------+---------+
|        0|           0|           0|       0|        0|     0|       374|       1|       374| 374|      365|          0|             0|      0|     0|   0|      0|      0|     179|      0|         0|      0|     229|        0|
+---------+------------+------------+--------+---------+------+----------+--------+-----

In [32]:
perc_missing_values= (
    weather.select([
        F.round(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c))/F.count(F.lit(1)),2)
        .alias(c) for c in cols
    ])
)
perc_missing_values.show()

+---------+------------+------------+--------+---------+------+----------+--------+----------+----+---------+-----------+--------------+-------+------+----+-------+-------+--------+-------+----------+-------+--------+---------+
|feelslike|feelslikemax|feelslikemin|humidity|moonphase|precip|precipprob|pressure|severerisk|snow|snowdepth|solarenergy|solarradiation|sunrise|sunset|temp|tempmax|tempmin|tzoffset|uvindex|visibility|winddir|windgust|windspeed|
+---------+------------+------------+--------+---------+------+----------+--------+----------+----+---------+-----------+--------------+-------+------+----+-------+-------+--------+-------+----------+-------+--------+---------+
|      0.0|         0.0|         0.0|     0.0|      0.0|   0.0|      0.94|     0.0|      0.94|0.94|     0.92|        0.0|           0.0|    0.0|   0.0| 0.0|    0.0|    0.0|    0.45|    0.0|       0.0|    0.0|    0.58|      0.0|
+---------+------------+------------+--------+---------+------+----------+--------+-----

In [28]:
# drop columns where missing values are more than 70%

weather= weather.drop('precipprob', 'snow', 'snowdepth')

if 'severerisk' in weather.columns:
    weather= weather.drop('severerisk')

weather.columns

['weather_date',
 'feelslike',
 'feelslikemax',
 'feelslikemin',
 'humidity',
 'moonphase',
 'precip',
 'pressure',
 'solarenergy',
 'solarradiation',
 'sunrise',
 'sunset',
 'temp',
 'tempmax',
 'tempmin',
 'tzoffset',
 'uvindex',
 'visibility',
 'winddir',
 'windgust',
 'windspeed']

In [29]:
weather= weather.repartition(10)

weather.write.parquet('s3a://hrc-de-data/processed/cycling-dimension/weather/', mode='overwrite')